# Distributed training with Vertex Reduction server

In [23]:
import os
import pprint
import sys
import time

from google.cloud import aiplatform
from google.cloud.aiplatform_v1beta1 import types

## Set up environment

In [37]:
PROJECT = 'jk-mlops-dev'
REGION = 'us-west1'
ARTIFACT_REPO = f'jk-docker-repo-{REGION}'

## Prepare  a training container

### Create a Dockerfile

In [38]:
BASE_IMAGE = 'gcr.io/deeplearning-platform-release/tf2-gpu.2-5'
MODEL_GARDEN_VERSION = '2.5.0'
TF_TEXT='2.5.0'
TRAIN_IMAGE = f'{REGION}-docker.pkg.dev/{PROJECT}/{ARTIFACT_REPO}/model_garden'

dockerfile = f'''
FROM {BASE_IMAGE}

RUN apt remove -y google-fast-socket && \
    echo "deb https://packages.cloud.google.com/apt google-fast-socket main" | tee /etc/apt/sources.list.d/google-fast-socket.list && \
    curl -s -L https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - && \
    apt update && apt install -y google-reduction-server

RUN pip install tf-models-official=={MODEL_GARDEN_VERSION} tensorflow-text=={TF_TEXT}

WORKDIR /

# Copies the trainer code to the docker image.
COPY trainer /trainer

ENTRYPOINT ["python"]
CMD ["-c", "print('Hello')"]
'''

with open('Dockerfile', 'w') as f:
    f.write(dockerfile)

### Build a container image

In [39]:
! docker build -t {TRAIN_IMAGE} .

Sending build context to Docker daemon  528.4kB
Step 1/7 : FROM gcr.io/deeplearning-platform-release/tf2-gpu.2-5
 ---> b963122c3c2c
Step 2/7 : RUN apt remove -y google-fast-socket &&     echo "deb https://packages.cloud.google.com/apt google-fast-socket main" | tee /etc/apt/sources.list.d/google-fast-socket.list &&     curl -s -L https://packages.cloud.google.com/apt/doc/apt-key.gpg | apt-key add - &&     apt update && apt install -y google-reduction-server
 ---> Using cache
 ---> b5c475a6b8e1
Step 3/7 : RUN pip install tf-models-official==2.5.0 tensorflow-text==2.5.0
 ---> Using cache
 ---> 13bdf2be97e5
Step 4/7 : WORKDIR /
 ---> Using cache
 ---> 8c4ebbb80601
Step 5/7 : COPY trainer /trainer
 ---> Using cache
 ---> 247e00d0adc6
Step 6/7 : ENTRYPOINT ["python"]
 ---> Using cache
 ---> 9a74954f3825
Step 7/7 : CMD ["-c", "print('Hello')"]
 ---> Using cache
 ---> 1ba3b2f26eb2
Successfully built 1ba3b2f26eb2
Successfully tagged us-west1-docker.pkg.dev/jk-mlops-dev/jk-docker-repo-us-west1/

### Push the container to Container Registry

In [40]:
! docker push {TRAIN_IMAGE}

Using default tag: latest
The push refers to repository [us-west1-docker.pkg.dev/jk-mlops-dev/jk-docker-repo-us-west1/model_garden]

e1fe4344: Preparing 
015e2c00: Preparing 
7f4f633d: Preparing 
961a296c: Preparing 
53abc6c2: Preparing 
3723ef37: Preparing 
0089a9c0: Preparing 
3e41a2c0: Preparing 
25162004: Preparing 
99d982dd: Preparing 
6603d114: Preparing 
c97a79f1: Preparing 
e02b8502: Preparing 
d34a65ac: Preparing 
ce22e436: Preparing 
7e013d33: Preparing 
aff4f6ee: Preparing 
e4ccb381: Preparing 
90ceec1e: Preparing 
0ab30137: Preparing 
ed8ae595: Preparing 
855df562: Preparing 
db3c5655: Preparing 
0a9a6a11: Preparing 
723ef37: Waiting g 
8f196cf4: Preparing 
01dbc7de: Preparing 
31d2d72b: Preparing 
e41a2c0: Waiting g 
5162004: Waiting g 
49f5bf51: Preparing 
9d982dd: Waiting g 
603d114: Waiting g 
dd81f9fa: Preparing 
e41a2c0: Pushed   4.927GB/4.801GBPushing  202.1MB/543.3MBPushing  456.5MB/4.801GBPushing    456MB/543.3MBPushing  327.5MB/742.5MBPushing  192.9MB/1.832GBPushi

## Submit Vertext Training jobs


### Define helper functions

In [41]:
def prepare_worker_pool_specs(
    image_uri,
    args,
    cmd, 
    replica_count=1,
    machine_type='n1-standard-4',
    accelerator_count=0,
    accelerator_type='ACCELERATOR_TYPE_UNSPECIFIED',
    reduction_server_count=0,
    reduction_server_machine_type='n1-highcpu-16',
    reduction_server_image_uri='us-docker.pkg.dev/vertex-ai-restricted/training/reductionserver:latest'
):

    if accelerator_count > 0:
        machine_spec = {
            'machine_type': machine_type,
            'accelerator_type': accelerator_type,
            'accelerator_count': accelerator_count,
        }
    else:
        machine_spec = {
            'machine_type': machine_type
        }
    
    container_spec = {
        'image_uri': image_uri,
        'args': args,
        'command': cmd,
    }
    
    chief_spec = {
        'replica_count': 1,
        'machine_spec': machine_spec,
        'container_spec': container_spec
    }

    worker_pool_specs = [chief_spec]
    if replica_count > 1:
        workers_spec = {
            'replica_count': replica_count - 1,
            'machine_spec': machine_spec,
            'container_spec': container_spec
        }
        worker_pool_specs.append(workers_spec)
        
    if reduction_server_count > 1:
        workers_spec = {
            'replica_count': reduction_server_count,
            'machine_spec': {
                'machine_type': reduction_server_machine_type,
            },
            'container_spec': {
                'image_uri': reduction_server_image_uri
            }
        }
        worker_pool_specs.append(workers_spec)
    
    return worker_pool_specs

### Prepare a worker pool specification

In [34]:
# Task config
MNLI_TRAIN_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record'
MNLI_VALID_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record'
BERT_HUB_URL = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'
job_name = "JOB_{}".format(time.strftime("%Y%m%d_%H%M%S"))
output_dir = f'gs://jk-vertex-demos/jobs'
model_dir = f'{output_dir}/{job_name}/model'
tfhub_cache_dir = f'{output_dir}/tfhub-cache'
config_file = 'trainer/glue_mnli_matched.yaml'
#mode = 'train_and_eval'
mode = 'train'
experiment = 'bert/sentence_prediction'

# Chief and workers config
machine_type = 'n1-standard-8'
accelerator_count = 1
accelerator_type = 'NVIDIA_TESLA_T4' # 16 Gps egress
all_reduce_alg = 'nccl'
strategy = 'multi_worker_mirrored'
replica_count = 2

# Reduction server config
reduction_server_count=0
reduction_server_machine_type='n1-highcpu-16' # 16 Gbps egress

# Trainer config
train_steps = 400
steps_per_loop = 100
summary_interval = 100
validation_interval = 600
checkpoint_interval = 600
global_batch_size = 32


params_override = [
    'task.train_data.input_path=' + MNLI_TRAIN_SPLIT,
    'task.validation_data.input_path=' + MNLI_VALID_SPLIT,
    'task.train_data.global_batch_size=' + str(global_batch_size),
    'task.validation_data.global_batch_size=' + str(global_batch_size),
    'task.hub_module_url=' + BERT_HUB_URL,
    'runtime.num_gpus=' + str(accelerator_count),
    'runtime.distribution_strategy=' + strategy,
    'runtime.all_reduce_alg=' + all_reduce_alg,
    'trainer.train_steps=' + str(train_steps),
    'trainer.steps_per_loop=' + str(steps_per_loop),
    'trainer.summary_interval=' + str(summary_interval),
    'trainer.validation_interval=' + str(validation_interval),
    'trainer.checkpoint_interval=' + str(checkpoint_interval),
]


cmd = [
    "python", "trainer/train.py"
]
args = [
    '--experiment=' + experiment,
    '--mode=' + mode,
    '--model_dir=' + model_dir,
    '--config_file=' + config_file,
    '--tfhub_cache_dir=' + tfhub_cache_dir,
    '--params_override=' + ','.join(params_override),
]

worker_pool_specs = prepare_worker_pool_specs(
    image_uri=TRAIN_IMAGE,
    args=args,
    cmd=cmd,
    replica_count=replica_count,
    machine_type=machine_type,
    accelerator_count=accelerator_count,
    accelerator_type=accelerator_type,
    reduction_server_count=reduction_server_count,
    reduction_server_machine_type=reduction_server_machine_type,
)

pp = pprint.PrettyPrinter()
print(pp.pformat(worker_pool_specs))

[{'container_spec': {'args': ['--experiment=bert/sentence_prediction',
                              '--mode=train',
                              '--model_dir=gs://jk-vertex-demos/jobs/JOB_20210611_171514/model',
                              '--config_file=trainer/glue_mnli_matched.yaml',
                              '--tfhub_cache_dir=gs://jk-vertex-demos/jobs/tfhub-cache',
                              '--params_override=task.train_data.input_path=gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record,task.validation_data.input_path=gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record,task.train_data.global_batch_size=32,task.validation_data.global_batch_size=32,task.hub_module_url=https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4,runtime.num_gpus=1,runtime.distribution_strategy=mirrored,runtime.all_reduce_alg=nccl,trainer.train_steps=400,trainer.steps_per_loop=100,trainer.summary_interval=100,trainer.validation_interval=600,trainer.checkpoint_interval=600'],
   

### Submit and monitor the job

In [35]:
STAGING_BUCKET = f'gs://jk-vertex-{REGION}'
aiplatform.init(
    project=PROJECT,
    location=REGION,
    staging_bucket=STAGING_BUCKET
)
REGION

'us-central1'

In [36]:
display_name = job_name

job = aiplatform.CustomJob(
    display_name=display_name,
    worker_pool_specs=worker_pool_specs,
)

job.run(sync=False)

INFO:google.cloud.aiplatform.jobs:Creating CustomJob
INFO:google.cloud.aiplatform.jobs:CustomJob created. Resource name: projects/895222332033/locations/us-central1/customJobs/4205592393824600064
INFO:google.cloud.aiplatform.jobs:To use this CustomJob in another session:
INFO:google.cloud.aiplatform.jobs:custom_job = aiplatform.CustomJob.get('projects/895222332033/locations/us-central1/customJobs/4205592393824600064')
INFO:google.cloud.aiplatform.jobs:View Custom Job:
https://console.cloud.google.com/ai/platform/locations/us-central1/training/4205592393824600064?project=895222332033
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/4205592393824600064 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/customJobs/4205592393824600064 current state:
JobState.JOB_STATE_PENDING
INFO:google.cloud.aiplatform.jobs:CustomJob projects/895222332033/locations/us-central1/

RuntimeError: Job failed with:
code: 3
message: "The replica workerpool0-0 exited with a non-zero status of 139(SIGSEGV). Termination reason: Error. To find out more about why your job exited please check the logs: https://console.cloud.google.com/logs/viewer?project=895222332033&resource=ml_job%2Fjob_id%2F4205592393824600064&advancedFilter=resource.type%3D%22ml_job%22%0Aresource.labels.job_id%3D%224205592393824600064%22"


In [ ]:
job.resource_name

In [ ]:
job.wait()

### Test the container image locally

In [ ]:
MNLI_TRAIN_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_train.tf_record'
MNLI_VALID_SPLIT = 'gs://jk-vertex-demos/datasets/MNLI/mnli_valid.tf_record'
BERT_HUB_URL = 'https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/4'

num_gpus = 2 
strategy = 'mirrored'
#strategy = 'multi_worker_mirrored'

params_override = [
    'task.train_data.input_path=' + MNLI_TRAIN_SPLIT,
    'task.validation_data.input_path=' + MNLI_VALID_SPLIT,
    'task.hub_module_url=' + BERT_HUB_URL,
    'runtime.num_gpus=' + str(num_gpus),
    'runtime.distribution_strategy=' + strategy,
]

params = ','.join(params_override)

In [ ]:
!docker run -it --rm --gpus all {TRAIN_IMAGE} trainer/train.py \
--experiment=bert/sentence_prediction \
--mode=train_and_eval \
--model_dir={STAGING_BUCKET}/test \
--config_file=trainer/glue_mnli_matched.yaml \
--params_override={params}  
